In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
from utils.data_ingestion_detached_house import (
    Dict,
    load_detached_house_data,
    calc_japan_detached_house_statistics,
    extract_pref_city_code,
    extract_house_distribution,
    calc_pref_city_distribution,
    tokyo_wards,
)

ftr_pref_muni = ['pref', 'muni']

In [4]:
# ============================================================================ #
# Detached house data
# ============================================================================ #

file_loc = "data/ref/e008_1.xlsx"

# load fine
df = load_detached_house_data(file_loc)	

# calculate the Japanese statistics
jp_dhouse_stat = calc_japan_detached_house_statistics(df)

# caculate the distribution on prefectures and cities
df_pref_distribution, df_city_distribution = calc_pref_city_distribution(df)

# DataFrame Index of the 21 cities
city_index_tuple = df_city_distribution.set_index(['pref', 'muni']).index.to_list()

In [5]:
print(
    f"There are {round(jp_dhouse_stat.total / 1_000_000, 1)} Million Detached Houses in Japan"
)
print(
    f"National Share of Old Detach House: {round(jp_dhouse_stat.old_share, 2)}%",
)

There are 26.8 Million Detached Houses in Japan
National Share of Old Detach House: 31.44%


In [6]:
# ============================================================================ #
# Load the municipal PV capacity and extract the data for 21 major cities
# ============================================================================ #

municipal_pv_cap_rpv = pd.read_csv("data/ref/municipal_pv_cap_rpv.csv")
rpv_cap_cities = municipal_pv_cap_rpv[
    municipal_pv_cap_rpv[["pref", "muni"]].apply(tuple, axis=1).isin(city_index_tuple)
]
rpv_cap_cities = rpv_cap_cities.drop(["address", "county"], axis=1).set_index(
    ["pref", "muni"]
)

rpv_cap_tokyo = municipal_pv_cap_rpv[municipal_pv_cap_rpv["muni"].isin(tokyo_wards)]
rpv_cap_tokyo = rpv_cap_tokyo.set_index("muni").drop(
    ["address", "pref", "county"], axis=1
)
rpv_cap_tokyo.index.name = None

rpv_cap_cities.loc[("東京都", "特別区部"), :] = rpv_cap_tokyo.sum()
rpv_cap_cities = rpv_cap_cities.astype(int)
rpv_cap_cities.reset_index()

,pref,muni,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,北海道,札幌市,20742,24027,26463,28171,30031,33820,37528,41996,47889,53593
1,宮城県,仙台市,46570,53042,59757,65247,70458,77265,81988,87336,94876,102329
2,埼玉県,さいたま市,25267,32737,39360,44507,49664,58038,64888,72592,82675,91901
3,千葉県,千葉市,16456,21157,25458,28811,32290,37558,42128,47810,54125,60183
4,神奈川県,横浜市,44178,55729,64723,70959,77479,88351,98288,109703,122152,134575
5,神奈川県,川崎市,12479,15826,18429,20575,22739,26155,29056,32584,36228,40226
6,神奈川県,相模原市,14899,19161,22628,25281,27736,31751,35413,39298,44143,48887
7,新潟県,新潟市,21157,24228,27018,29561,32133,35156,37536,40365,43626,47210
8,静岡県,静岡市,54116,60883,66842,72138,76856,83597,89005,95303,103054,110141
9,静岡県,浜松市,80094,89216,98946,106284,112431,121969,130450,140099,151100,161435


In [7]:
# ============================================================================ #
# Load the number of rooftop PV and extract the data for 21 major cities
# ============================================================================ #

municipal_pv_no_rpv = pd.read_csv("data/ref/municipal_pv_no_rpv.csv")
rpv_no_cities = municipal_pv_no_rpv[
    municipal_pv_no_rpv[["pref", "muni"]].apply(tuple, axis=1).isin(city_index_tuple)
]
rpv_no_cities = rpv_no_cities.drop(["address", "county"], axis=1).set_index(
    ["pref", "muni"]
)

rpv_no_tokyo = municipal_pv_no_rpv[municipal_pv_no_rpv["muni"].isin(tokyo_wards)]
rpv_no_tokyo = rpv_no_tokyo.set_index("muni").drop(
    ["address", "pref", "county"], axis=1
)
rpv_no_tokyo.index.name = None

rpv_no_cities.loc[("東京都", "特別区部"), :] = rpv_no_tokyo.sum()
rpv_no_cities = rpv_no_cities.astype(int)
rpv_no_cities.reset_index()

,pref,muni,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,北海道,札幌市,5284,5960,6471,6847,7216,7915,8578,9341,10369,11333
1,宮城県,仙台市,11732,13243,14752,15951,17075,18531,19539,20628,22057,23605
2,埼玉県,さいたま市,6441,8197,9761,10910,12079,13979,15499,17236,19610,21966
3,千葉県,千葉市,4155,5259,6228,6951,7708,8833,9810,10988,12268,13593
4,神奈川県,横浜市,12121,15113,17223,18666,20257,22842,25115,27801,30724,33729
5,神奈川県,川崎市,3338,4161,4799,5298,5834,6682,7339,8159,9060,10186
6,神奈川県,相模原市,3834,4831,5596,6167,6719,7608,8400,9230,10239,11303
7,新潟県,新潟市,5255,5882,6430,6910,7395,7998,8464,8978,9567,10233
8,静岡県,静岡市,13408,14917,16196,17341,18335,19757,20798,21985,23450,24800
9,静岡県,浜松市,19184,21072,23050,24570,25825,27761,29366,31035,32909,34647


In [8]:
# ============================================================================ #
# Average size of rooftop PV
# ============================================================================ #
(rpv_cap_cities/rpv_no_cities).reset_index()

,pref,muni,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,北海道,札幌市,3.925435,4.031376,4.089476,4.114357,4.161724,4.272900,4.374913,4.495878,4.618478,4.728933
1,宮城県,仙台市,3.969485,4.005286,4.050773,4.090465,4.126384,4.169500,4.196121,4.233857,4.301401,4.335056
2,埼玉県,さいたま市,3.922838,3.993778,4.032374,4.079468,4.111599,4.151799,4.186593,4.211650,4.215961,4.183784
3,千葉県,千葉市,3.960529,4.023008,4.087669,4.144871,4.189154,4.252010,4.294393,4.351110,4.411885,4.427499
4,神奈川県,横浜市,3.644749,3.687488,3.757940,3.801511,3.824801,3.867919,3.913518,3.946009,3.975784,3.989890
5,神奈川県,川崎市,3.738466,3.803413,3.840175,3.883541,3.897669,3.914247,3.959122,3.993627,3.998675,3.949146
6,神奈川県,相模原市,3.886020,3.966260,4.043603,4.099400,4.127995,4.173370,4.215833,4.257638,4.311261,4.325135
7,新潟県,新潟市,4.026070,4.119007,4.201866,4.278003,4.345233,4.395599,4.434783,4.495990,4.560050,4.613505
8,静岡県,静岡市,4.036098,4.081451,4.127068,4.159968,4.191764,4.231260,4.279498,4.334910,4.394627,4.441169
9,静岡県,浜松市,4.175042,4.233865,4.292668,4.325763,4.353572,4.393538,4.442212,4.514226,4.591449,4.659422


In [9]:
# ============================================================================ #
# Municipal Demand
# ============================================================================ #

municipal_demand = pd.read_csv("data/ref/muni_demand_fy2022.csv")

demand_cities = municipal_demand[
    municipal_demand[["pref", "muni"]].apply(tuple, axis=1).isin(city_index_tuple)
]
demand_cities = demand_cities.set_index(['pref', 'muni'])
demand_tokyo = municipal_demand[municipal_demand['muni'].isin(tokyo_wards)].set_index('muni').drop('pref', axis=1)
demand_tokyo.index.name = None

demand_cities.loc[("東京都", "特別区部"), :] = demand_tokyo.sum()
demand_cities = demand_cities.astype(int)
demand_cities.reset_index()

,pref,muni,2022.4,2022.5,2022.6,2022.7,2022.8,2022.9,2022.10,2022.11,2022.12,2023.1,2023.2,2023.3,total
0,北海道,札幌市,723748,632649,562211,640183,645979,608037,599922,668017,821405,1006851,883062,776410,8568474
1,宮城県,仙台市,536961,445147,451933,490997,507391,479034,428496,433848,537607,644690,601565,537866,6095535
2,埼玉県,さいたま市,432678,380648,407057,508913,555444,487621,403610,395128,454323,538039,541275,435168,5539904
3,千葉県,千葉市,393125,343364,355502,441599,493610,436540,380543,388847,424022,474402,482620,382477,4996651
4,神奈川県,横浜市,1308569,1184707,1256216,1495924,1655722,1535575,1297307,1232401,1348934,1561383,1573336,1341689,16791763
5,神奈川県,川崎市,920480,870072,874173,972219,1040896,996377,897683,856831,920435,982253,980306,903233,11214964
6,神奈川県,相模原市,295397,259043,277134,324994,348429,326205,282894,278917,297567,342132,349602,304380,3686694
7,新潟県,新潟市,341960,315702,297863,356165,381609,348302,294422,306807,358760,423083,407256,352335,4184264
8,静岡県,静岡市,308510,280246,279434,333554,363710,358089,303101,275973,290631,366630,337426,300034,3797338
9,静岡県,浜松市,389076,348583,358173,422203,457699,462637,396960,352944,377578,466488,441504,401182,4875027


In [10]:
# ============================================================================ #
# Monthly Municipal PV Generation [kWh/kw]
# ============================================================================ #

municipal_pv_output = pd.read_csv('data/ref/municipal_monthly_pv_output.csv')
municipal_pv_output = municipal_pv_output.drop('county', axis=1)
pv_out_cities = municipal_pv_output[
    municipal_pv_output[["pref", "muni"]].apply(tuple, axis=1).isin(city_index_tuple)
]
pv_out_cities = pv_out_cities.set_index(['pref', 'muni'])

pv_out_tokyo = municipal_pv_output[municipal_pv_output['muni'].isin(tokyo_wards)].set_index('muni').drop('pref', axis=1)
pv_out_tokyo.index.name = None

pv_out_cities.loc[("東京都", "特別区部"), :] = pv_out_tokyo.mean()

pv_out_cities['total'] = pv_out_cities.sum(axis=1).to_frame('pv_out')

pv_out_cities.round(2).reset_index()

,pref,muni,1,2,3,4,5,6,7,8,9,10,11,12,total
0,北海道,札幌市,88.01,111.10,149.12,147.57,138.94,109.87,123.51,114.76,87.62,104.32,75.00,82.18,1332.00
1,宮城県,仙台市,84.96,97.71,122.12,127.67,115.42,113.21,63.27,112.08,85.03,87.37,96.98,84.22,1190.05
2,埼玉県,さいたま市,95.84,116.10,120.34,121.23,107.47,91.51,58.33,129.47,81.80,73.12,98.44,100.40,1194.05
3,千葉県,千葉市,88.70,112.10,117.17,124.09,118.89,101.31,76.85,143.25,95.66,77.41,89.26,95.51,1240.19
4,神奈川県,横浜市,91.14,109.96,115.90,119.57,113.26,92.74,65.88,135.14,87.14,74.91,91.22,95.61,1192.49
5,神奈川県,川崎市,89.62,110.44,115.46,119.98,112.88,93.97,68.08,137.09,88.67,74.96,89.47,94.58,1195.21
6,神奈川県,相模原市,94.37,110.83,117.97,121.84,115.36,92.84,64.69,133.95,86.61,76.30,94.57,99.07,1208.39
7,新潟県,新潟市,66.83,84.19,116.36,127.18,124.30,117.75,81.18,121.57,107.08,91.15,79.89,54.26,1171.74
8,静岡県,静岡市,111.80,120.99,134.21,150.85,133.27,102.41,69.06,137.66,91.97,95.25,112.73,122.56,1382.77
9,静岡県,浜松市,107.86,115.07,129.75,150.68,126.39,97.92,67.13,133.49,92.04,99.20,110.43,119.29,1349.24


In [11]:
# ============================================================================ #
# CONSOLIDATE DATA
# ============================================================================ #

# ============================================================================ #
# Rooftop PV Capacity
# ============================================================================ #

renamed_col = {c: f"rpv_cap_{c}" for c in rpv_cap_cities.columns}
df_clean = rpv_cap_cities.rename(columns=renamed_col).copy()

# ============================================================================ #
# No of Rooftop Installations
# ============================================================================ #

renamed_col = {c: f"rpv_no_{c}" for c in rpv_no_cities.columns}
df_temp = rpv_no_cities.rename(columns=renamed_col)
df_clean = pd.merge(df_clean, df_temp, how="right", left_index=True, right_index=True)

# ============================================================================ #
# PV Output
# ============================================================================ #

df_clean["pv_out"] = pv_out_cities["total"]

# ============================================================================ #
# Municipal Demand
# ============================================================================ #

df_clean["dmd_muni"] = demand_cities["total"]

# ============================================================================ #
# Prefectural Residential Demand
# ============================================================================ #

df_pref_energy_share = pd.read_csv("data/ref/pref_electricity_share_2021.csv")
pref_share_residential_dict = df_pref_energy_share.set_index("pref")[
    "share_residential"
].to_dict()
df_clean["pref_res_ene_share"] = df_clean.index.get_level_values("pref").map(
    pref_share_residential_dict
)

# ============================================================================ #
# City Detached House Distribution
# ============================================================================ #

df_temp = df_city_distribution.copy()
df_temp = df_temp.set_index(["pref", "muni"])
impt_col = ["old", "new"]
new_col = [f"dhouse_{ind}" for ind in impt_col]

df_temp = df_temp[impt_col]
df_temp.columns = new_col
df_clean[new_col] = df_temp[new_col]

# ============================================================================ #
# Save
# ============================================================================ #
if True:
    df_clean.reset_index().to_csv(
        "data/gen/jp_major_cities_rtpv_dataset.csv", index=False
    )

df_clean

rpv_cap_2014  rpv_cap_2015  rpv_cap_2016  rpv_cap_2017  \
pref muni                                                            
北海道  札幌市           20742         24027         26463         28171   
宮城県  仙台市           46570         53042         59757         65247   
埼玉県  さいたま市         25267         32737         39360         44507   
千葉県  千葉市           16456         21157         25458         28811   
神奈川県 横浜市           44178         55729         64723         70959   
     川崎市           12479         15826         18429         20575   
     相模原市          14899         19161         22628         25281   
新潟県  新潟市           21157         24228         27018         29561   
静岡県  静岡市           54116         60883         66842         72138   
     浜松市           80094         89216         98946        106284   
愛知県  名古屋市          68584         79741         90582         98875   
京都府  京都市           39540         44856         48955         52423   
大阪府  大阪市           33504         37964         42356         45158   
     堺市            41749         46307         49883         52518   
兵庫県  神戸市           55588         61615         67005         70592   
岡山県  岡山市           24673         30302         37118         42719   
広島県  広島市           22659         29232         35519         40976   
福岡県  北九州市          49174         55397         60506         65112   
     福岡市           50379         56699         62632         67465   
熊本県  熊本市           29778         38205         44088         49858   
東京都  特別区部         140713        155108        166941        175696   

            rpv_cap_2018  rpv_cap_2019  rpv_cap_2020  rpv_cap_2021  \
pref muni                                                            
北海道  札幌市           30031         33820         37528         41996   
宮城県  仙台市           70458         77265         81988         87336   
埼玉県  さいたま市         49664         58038         64888         72592   
千葉県  千葉市           32290         37558         42128         47810   
神奈川県 横浜市           77479         88351         98288        109703   
     川崎市           22739         26155         29056         32584   
     相模原市          27736         31751         35413         39298   
新潟県  新潟市           32133         35156         37536         40365   
静岡県  静岡市           76856         83597         89005         95303   
     浜松市          112431        121969        130450        140099   
愛知県  名古屋市         106600        120280        132587        146277   
京都府  京都市           55960         59468         62902         66651   
大阪府  大阪市           47790         51543         55060         59132   
     堺市            55002         58050         61351         64937   
兵庫県  神戸市           74396         79847         84516         90028   
岡山県  岡山市           47683         55058         62281         70181   
広島県  広島市           46504         54764         61695         68438   
福岡県  北九州市          69237         76122         81321         86829   
     福岡市           71999         79238         84454         90176   
熊本県  熊本市           57792         69943         78939         87819   
東京都  特別区部         184074        195522        205905        218577   

            rpv_cap_2022  rpv_cap_2023  ...  rpv_no_2019  rpv_no_2020  \
pref muni                               ...                             
北海道  札幌市           47889         53593  ...         7915         8578   
宮城県  仙台市           94876        102329  ...        18531        19539   
埼玉県  さいたま市         82675         91901  ...        13979        15499   
千葉県  千葉市           54125         60183  ...         8833         9810   
神奈川県 横浜市          122152        134575  ...        22842        25115   
     川崎市           36228         40226  ...         6682         7339   
     相模原市          44143         48887  ...         7608         8400   
新潟県  新潟市           43626         47210  ...         7998         8464   
静岡県  静岡市          103054        110141  ...        19757    

In [12]:
# ============================================================================ #
# Extract hourly demand from EPCO disclosure data
# ============================================================================ #

epco_data_filename = "data/ref/japan_epco_clean_fy2022.csv"
if os.path.exists(epco_data_filename):
    epco_hourly_fy2022 = pd.read_csv(epco_data_filename, header=[0, 1], index_col=0)
    epco_hourly_fy2022.index = pd.to_datetime(epco_hourly_fy2022.index)

    epco_hourly_demand_fy2022 = epco_hourly_fy2022.xs(key="Demand", axis=1, level=1)
    epco_hourly_demand_fy2022.index.name = None
    epco_hourly_demand_fy2022.to_csv("data/ref/epco_hourly_demand_fy2022.csv")